In [4]:
print('hello world3f2')


hello world3f2


In [5]:
#### Import Dependencies ####
import nemo.collections.asr as nemo_asr
from nemo.utils import exp_manager
import pytorch_lightning
from omegaconf import OmegaConf, open_dict
from nemo.core import adapter_mixins
from pytorch_lightning import Trainer
from nemo.collections.common.parts.adapter_modules import LinearAdapterConfig
import torch

from pytorch_lightning.loggers import WandbLogger

[NeMo W 2023-04-01 17:04:55 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


In [6]:
import wandb
wandb.login()

### API KEY : 185835c122601380ffad33b46fc36065559c4476

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: srksaurbzz. Use `wandb login --relogin` to force relogin


True

In [7]:
####cONSTANTS #####
TRAIN_MANIFEST = '/home/saurabh/manifest/train.json'
VALID_MANIFEST = '/home/saurabh/manifest/valid.json'

# Model save path
model_path_pretrained = '/home/saurabh/pretrained_models/Conformer_hi/stt_hi_conformer_ctc_medium.nemo'
model_32 = '/home/saurabh/experiments/Telegu_IS_32/checkpoints/teleguISTrain_scratch_32.nemo'
model_64 = '/home/saurabh/experiments/Telegu_IS_64/checkpoints/teleguISTrain_scratch.nemo'

model_path = model_path_pretrained

accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
# accelerator = 'cpu'
max_steps = -1
max_epochs = 10

#Spectogram augmentation
lr = 5
weight_decay = 0
warmup_steps = 10000


## Adapter 
adapter_name = 'Confomer_telegu'
adapter_dim = 64
adapter_activtion = 'swish'
adapter_norm_position = 'pre'


#EXP
a64 = 'Telegu_IS_64'
a32 = 'Telegu_IS_32' 
a128 = 'Telegu_IS_128' 

exp_direc=f'/home/saurabh/experiments' # Don't change
name_exp = a64 # where it will be saved
#wandb_karwads
name_wandb='run01'
project_wandb = a64



#### Load model #####
model= nemo_asr.models.ASRModel.restore_from(model_path)


##### adpter metadata #####
def update_model_config_to_support_adapter(model_cfg):
  with open_dict(model_cfg):
    adapter_metadata = adapter_mixins.get_registered_adapter(model_cfg.encoder._target_)
    if adapter_metadata is not None:
      model_cfg.encoder._target_ = adapter_metadata.adapter_class_path
  
  print("Updated encoder _target_ model: ", model_cfg.encoder._target_)
  return model_cfg

cfg = update_model_config_to_support_adapter(model.cfg)
model = nemo_asr.models.ASRModel.restore_from(model_path, override_config_path=cfg)


##### Trainer #####
trainer = Trainer(devices=1, accelerator=accelerator, max_steps=max_steps,
                  enable_checkpointing=False, logger=False,
                  log_every_n_steps=5, check_val_every_n_epoch=3, 
                  max_epochs=max_epochs)

model.set_trainer(trainer)


##### Dataloader for updater #######
with open_dict(model.cfg):
  model.cfg.train_ds.manifest_filepath = TRAIN_MANIFEST
  model.cfg.train_ds.is_tarred = False
  # model.cfg.train_ds.tarred_audio_filepaths = None 
  model.cfg.train_ds.num_workers = 2
  model.cfg.train_ds.batch_size = 16

  model.cfg.validation_ds.manifest_filepath = VALID_MANIFEST
  model.cfg.validation_ds.num_workers = 2
  model.cfg.validation_ds.batch_size = 16

model.setup_training_data(model.cfg.train_ds)
model.setup_multiple_validation_data(model.cfg.validation_ds)
model.setup_multiple_test_data(model.cfg.validation_ds)


adapter_cfg = LinearAdapterConfig(
    in_features=model.cfg.encoder.d_model,
    dim=adapter_dim,
    activation=adapter_activtion,
    norm_position=adapter_norm_position,
)

model.add_adapter(name=adapter_name, cfg=adapter_cfg)
model.set_enabled_adapters(enabled=False) # disable all adapters
model.set_enabled_adapters(name=adapter_name, enabled=True)
model.freeze()
model.unfreeze_enabled_adapters()


##### Experiment Manager #######
exp_config = exp_manager.ExpManagerConfig(
    exp_dir=exp_direc,
    name=name_exp,
    
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor='val_wer',
        mode='min',
        always_save_nemo=True,
        save_best_model=True,
    ),
    
    resume_if_exists=True,
    resume_ignore_no_checkpoint=True,
    create_tensorboard_logger= True,
    create_checkpoint_callback= True,
  
    create_wandb_logger=True,
    wandb_logger_kwargs={
        'name':name_wandb, 
        'project':project_wandb
    },
)
exp_config=OmegaConf.structured(exp_config)
logdir = exp_manager.exp_manager(trainer, exp_config)

[NeMo I 2023-04-01 17:05:01 mixins:170] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2023-04-01 17:05:01 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /code/03_manifest/ulca-train-v3.json
    sample_rate: 16000
    batch_size: 64
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    
[NeMo W 2023-04-01 17:05:01 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /code/manifest/ulca-eval.json
    sample_rate: 16000
    batch_size: 4
    shuffle: false
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    
[NeMo W 2023-04-01 17:05:01 modelPT:174

[NeMo I 2023-04-01 17:05:01 features:287] PADDING: 0
[NeMo I 2023-04-01 17:05:08 save_restore_connector:247] Model EncDecCTCModelBPE was successfully restored from /home/saurabh/pretrained_models/Conformer_hi/stt_hi_conformer_ctc_medium.nemo.
Updated encoder _target_ model:  nemo.collections.asr.modules.conformer_encoder.ConformerEncoderAdapter
[NeMo I 2023-04-01 17:05:09 mixins:170] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2023-04-01 17:05:09 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /code/03_manifest/ulca-train-v3.json
    sample_rate: 16000
    batch_size: 64
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    
[NeMo W 2023-04-01 17:05:09 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /code/manifest/ulca-eval.json
    sample_rate: 16000
    batch_size: 4
    shuffle: false
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    
[NeMo W 2023-04-01 17:05:09 modelPT:174

[NeMo I 2023-04-01 17:05:09 features:287] PADDING: 0
[NeMo I 2023-04-01 17:05:10 save_restore_connector:247] Model EncDecCTCModelBPE was successfully restored from /home/saurabh/pretrained_models/Conformer_hi/stt_hi_conformer_ctc_medium.nemo.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[NeMo I 2023-04-01 17:05:11 collections:193] Dataset loaded with 12657 files totalling 17.58 hours
[NeMo I 2023-04-01 17:05:11 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-01 17:05:12 collections:193] Dataset loaded with 3207 files totalling 4.45 hours
[NeMo I 2023-04-01 17:05:12 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-01 17:05:12 collections:193] Dataset loaded with 3207 files totalling 4.45 hours
[NeMo I 2023-04-01 17:05:12 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-01 17:05:12 adapter_mixins:712] Setting adapter 'Confomer_telegu' status : Enabled = False
[NeMo I 2023-04-01 17:05:12 adapter_mixins:727] Setting adapter 'Confomer_telegu' status : Enabled = True
[NeMo I 2023-04-01 17:05:12 adapter_mixins:398] Froze module encoder.layers.0.conv.batch_norm: BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
[NeMo I 2023-04-01 17:05:12 adapter_mixins:398] Froze

[NeMo W 2023-04-01 17:05:12 exp_manager:710] No version folders would be created under the log folder as 'resume_if_exists' is enabled.


[NeMo I 2023-04-01 17:05:12 exp_manager:593] Resuming from /home/saurabh/experiments/Telegu_IS_64/checkpoints/teleguISTrain_scratch--val_wer=0.6287-epoch=300-last.ckpt
[NeMo I 2023-04-01 17:05:12 exp_manager:373] Experiments will be logged at /home/saurabh/experiments/Telegu_IS_64
[NeMo I 2023-04-01 17:05:12 exp_manager:791] TensorboardLogger has been set up


[NeMo I 2023-04-01 17:05:14 exp_manager:806] WandBLogger has been set up


In [8]:
model.summarize()

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConformerEncoderAdapter           | 31.1 M
2 | decoder           | ConvASRDecoder                    | 33.2 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WERBPE                            | 0     
------------------------------------------------------------------------
599 K     Trainable params
30.5 M    Non-trainable params
31.1 M    Total params
124.551   Total estimated model params size (MB)

In [ ]:
trainer.fit(model)

In [ ]:
save_model_path = '/home/saurabh/saved_model/Telugu_IS128.nemo'
model.save_to(save_model_path)